# Mining ASTER Analysis

## Objectives
- Alteration mapping (Clay, Iron, Carbonates)
- Mineralisation analysis
- Temporal comparison (2001-2006 vs 2023-2024)
- Integration with SRTM 30 DEM

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m


In [56]:
pip install geemap earthengine-api numpy

Note: you may need to restart the kernel to use updated packages.


In [57]:
# Import required libraries
import geemap
import ee
import numpy as np
from datetime import datetime

# Initialize Earth Engine with Project ID
# IMPORTANT: Replace 'your-project-id' with your actual Google Cloud Project ID
# To find your Project ID:
# 1. Go to https://console.cloud.google.com/
# 2. Click on the project dropdown at the top
# 3. Copy the Project ID (e.g., 'ee-your-username' or 'mineral-exploration-456')

PROJECT_ID = 'ee-okwaretom12'  # <-- REPLACE THIS WITH YOUR PROJECT ID

try:
    # Try to initialize with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print("Earth Engine not initialized. Starting authentication...")
    print("Please follow the authentication steps:")
    print("1. A browser window will open")
    print("2. Sign in with your Google account")
    print("3. Grant permissions to Earth Engine")
    print("4. Copy the authorization code and paste it when prompted")
    
    # Authenticate (this will open a browser for first-time users)
    ee.Authenticate()
    
    # Initialize after authentication with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized successfully with project: {PROJECT_ID}!")

# Initialize geemap
Map = geemap.Map()
print("Libraries imported and geemap initialized successfully!")


Earth Engine initialized with project: ee-okwaretom12
Libraries imported and geemap initialized successfully!


## Define Study Area


In [58]:
# Define study area coordinates (WGS1984)
# Convert DMS to decimal degrees
coords = [
    [29.575, 4.766667],  # Point 1: 29°34'30"E, 04°46'0"N
    [29.279167, 4.766667],  # Point 2: 29°16'45"E, 04°46'0"N
    [29.279167, 5.091667],  # Point 3: 29°16'45"E, 05°05'30"N
    [29.575, 5.091667],  # Point 4: 29°34'30"E, 05°05'30"N
]

# Create polygon
study_area = ee.Geometry.Polygon([coords])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

# Print study area info (without .getInfo() to avoid blocking)
print("Study area created with coordinates:")
print("  Point 1: 29°34'30\"E, 04°46'0\"N")
print("  Point 2: 29°16'45\"E, 04°46'0\"N")
print("  Point 3: 29°16'45\"E, 05°05'30\"N")
print("  Point 4: 29°34'30\"E, 05°05'30\"N")
print("  Buffer: 20m")

# Add to map
Map.addLayer(study_area_buffered, {'color': 'red'}, 'Study Area (20m buffer)')
Map.centerObject(study_area_buffered, 10)
Map


Study area created with coordinates:
  Point 1: 29°34'30"E, 04°46'0"N
  Point 2: 29°16'45"E, 04°46'0"N
  Point 3: 29°16'45"E, 05°05'30"N
  Point 4: 29°34'30"E, 05°05'30"N
  Buffer: 20m


Map(center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position', 'transparent_b…

In [59]:
# Calculate visualization parameters (data-driven ranges)
# These functions calculate optimal min/max values using percentiles (2nd-98th by default)
# to exclude outliers and improve visualization contrast

def calculate_viz_params(image, band_name, study_area, percentile_low=2, percentile_high=98):
    """
    Calculate optimal min/max values for visualization using percentiles
    Excludes outliers for better contrast
    
    Args:
        image: Earth Engine Image
        band_name: Name of the band to visualize
        study_area: Geometry for calculating statistics
        percentile_low: Lower percentile (default: 2)
        percentile_high: Upper percentile (default: 98)
    
    Returns:
        Dictionary with 'min' and 'max' values
    
    Note: Earth Engine returns percentile keys as '{band_name}_p{percentile}'
    """
    # Select the band to ensure we're working with the right band
    band_image = image.select(band_name)
    
    # Calculate percentiles
    stats = band_image.reduceRegion(
        reducer=ee.Reducer.percentile([percentile_low, percentile_high]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    # Earth Engine returns keys as '{band_name}_p{percentile}'
    # Construct the keys with the band name
    min_key = f'{band_name}_p{percentile_low}'
    max_key = f'{band_name}_p{percentile_high}'
    
    return {
        'min': stats.get(min_key),
        'max': stats.get(max_key)
    }

def calculate_rgb_viz_params(rgb_image, bands, study_area, percentile_low=2, percentile_high=98):
    """
    Calculate visualization parameters for RGB composite
    Returns min/max that work for all three bands
    
    Args:
        rgb_image: Earth Engine Image with RGB bands
        bands: List of band names (e.g., ['R', 'G', 'B'] or ['B3N', 'B02', 'B01'])
        study_area: Geometry for calculating statistics
        percentile_low: Lower percentile (default: 2)
        percentile_high: Upper percentile (default: 98)
    
    Returns:
        Dictionary with 'min' and 'max' values for RGB visualization
    """
    # Calculate percentiles for each band
    stats = rgb_image.select(bands).reduceRegion(
        reducer=ee.Reducer.percentile([percentile_low, percentile_high]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    # Get min and max for each band (construct keys with band names)
    # Use the first band to initialize
    first_band = bands[0]
    overall_min = stats.get(f'{first_band}_p{percentile_low}')
    overall_max = stats.get(f'{first_band}_p{percentile_high}')
    
    # Find overall min and max across all bands (server-side)
    for band in bands[1:]:
        band_min = stats.get(f'{band}_p{percentile_low}')
        band_max = stats.get(f'{band}_p{percentile_high}')
        overall_min = ee.Number(overall_min).min(ee.Number(band_min))
        overall_max = ee.Number(overall_max).max(ee.Number(band_max))
    
    return {
        'min': overall_min,
        'max': overall_max
    }

print("Visualization parameter calculation functions loaded!")
print("  - calculate_viz_params: For single-band images (2nd-98th percentile)")
print("  - calculate_rgb_viz_params: For RGB composites (2nd-98th percentile)")


Visualization parameter calculation functions loaded!
  - calculate_viz_params: For single-band images (2nd-98th percentile)
  - calculate_rgb_viz_params: For RGB composites (2nd-98th percentile)


## Load SRTM 30 DEM


In [60]:
# Load SRTM 30m DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)

# Calculate slope and aspect for lineament analysis
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
hillshade = ee.Terrain.hillshade(elevation)

# Calculate data-driven visualization parameters (2nd-98th percentile)
elevation_viz = calculate_viz_params(elevation, 'elevation', study_area_buffered)
slope_viz = calculate_viz_params(slope, 'slope', study_area_buffered)
hillshade_viz = calculate_viz_params(hillshade, 'hillshade', study_area_buffered)

# Add to map with data-driven ranges
Map.addLayer(elevation, {
    'min': elevation_viz['min'],
    'max': elevation_viz['max'],
    'palette': ['blue', 'green', 'yellow', 'red']
}, 'SRTM Elevation')
Map.addLayer(slope, {
    'min': slope_viz['min'],
    'max': slope_viz['max'],
    'palette': ['white', 'brown']
}, 'Slope')
Map.addLayer(hillshade, {
    'min': hillshade_viz['min'],
    'max': hillshade_viz['max']
}, 'Hillshade', False)

print("SRTM DEM loaded successfully!")
Map


SRTM DEM loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Load ASTER Data - Period 1 (2001-2006)


In [61]:
# Load ASTER L1T data for period 2001-2006
aster_collection_2001_2006 = ee.ImageCollection('ASTER/AST_L1T_003') \
    .filterBounds(study_area_buffered) \
    .filterDate('2001-01-01', '2006-12-31') \
    .filter(ee.Filter.lt('CLOUDCOVER', 20))  # Filter clouds

print(f"Number of ASTER images (2001-2006): {aster_collection_2001_2006.size().getInfo()}")

# Create median composite to reduce noise
aster_2001_2006 = aster_collection_2001_2006.median().clip(study_area_buffered)

# ASTER bands: B01-B09 (VNIR and SWIR)
# B01: 0.52-0.60 μm (Green)
# B02: 0.63-0.69 μm (Red)
# B3N: 0.76-0.86 μm (NIR)
# B04: 1.60-1.70 μm (SWIR)
# B05: 2.145-2.185 μm (SWIR)
# B06: 2.185-2.225 μm (SWIR)
# B07: 2.235-2.285 μm (SWIR)
# B08: 2.295-2.365 μm (SWIR)
# B09: 2.360-2.430 μm (SWIR)

# Calculate data-driven visualization parameters for RGB bands (2nd-98th percentile)
# Note: calculate_rgb_viz_params is already defined in Cell 5
aster_rgb_viz_2001 = calculate_rgb_viz_params(aster_2001_2006, ['B3N', 'B02', 'B01'], study_area_buffered)

# Add RGB composite with data-driven range
Map.addLayer(aster_2001_2006, {
    'bands': ['B3N', 'B02', 'B01'],
    'min': aster_rgb_viz_2001['min'],
    'max': aster_rgb_viz_2001['max']
}, 'ASTER RGB (2001-2006)')

Map


Number of ASTER images (2001-2006): 13


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Load ASTER Data - Period 2 (2023-2024)


In [62]:
# Load ASTER L1T data for period 2023-2024
aster_collection_2023_2024 = ee.ImageCollection('ASTER/AST_L1T_003') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDCOVER', 20))

print(f"Number of ASTER images (2023-2024): {aster_collection_2023_2024.size().getInfo()}")

# Create median composite
aster_2023_2024 = aster_collection_2023_2024.median().clip(study_area_buffered)

# Calculate data-driven visualization parameters for RGB bands (2nd-98th percentile)
aster_rgb_viz_2023 = calculate_rgb_viz_params(aster_2023_2024, ['B3N', 'B02', 'B01'], study_area_buffered)

# Add RGB composite with data-driven range
Map.addLayer(aster_2023_2024, {
    'bands': ['B3N', 'B02', 'B01'],
    'min': aster_rgb_viz_2023['min'],
    'max': aster_rgb_viz_2023['max']
}, 'ASTER RGB (2023-2024)')

Map


Number of ASTER images (2023-2024): 24


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Alteration Mapping - Clay Minerals


In [63]:
# Clay Mineral Index (CMI) using ASTER bands
# Clay minerals have strong absorption in SWIR bands
# CMI = (B06 + B07) / (B05 + B08)

def calculate_clay_index(image):
    """Calculate Clay Mineral Index"""
    b05 = image.select('B05').float()
    b06 = image.select('B06').float()
    b07 = image.select('B07').float()
    b08 = image.select('B08').float()
    
    # Clay Mineral Index
    clay_index = b06.add(b07).divide(b05.add(b08)).rename('Clay_Index')
    
    # AlOH Group Index (for clay detection)
    aloh_index = b05.divide(b06).rename('AlOH_Index')
    
    # Kaolinite Index
    kaolinite_index = b07.divide(b05).rename('Kaolinite_Index')
    
    return image.addBands([clay_index, aloh_index, kaolinite_index])

# Apply to both time periods
aster_2001_2006_clay = calculate_clay_index(aster_2001_2006)
aster_2023_2024_clay = calculate_clay_index(aster_2023_2024)

# Calculate data-driven visualization parameters (2nd-98th percentile)
clay_viz_2001 = calculate_viz_params(aster_2001_2006_clay, 'Clay_Index', study_area_buffered)
clay_viz_2023 = calculate_viz_params(aster_2023_2024_clay, 'Clay_Index', study_area_buffered)

# Visualize clay indices with data-driven ranges
Map.addLayer(aster_2001_2006_clay.select('Clay_Index'), {
    'min': clay_viz_2001['min'],
    'max': clay_viz_2001['max'],
    'palette': ['blue', 'cyan', 'yellow', 'red']
}, 'Clay Index (2001-2006)')

Map.addLayer(aster_2023_2024_clay.select('Clay_Index'), {
    'min': clay_viz_2023['min'],
    'max': clay_viz_2023['max'],
    'palette': ['blue', 'cyan', 'yellow', 'red']
}, 'Clay Index (2023-2024)')

print("Clay mineral indices calculated!")
Map


Clay mineral indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Alteration Mapping - Iron Oxides


In [64]:
# Iron Oxide Index using ASTER bands
# Iron oxides have strong absorption in visible bands
# Iron Index = B02 / B01

def calculate_iron_index(image):
    """Calculate Iron Oxide Index"""
    b01 = image.select('B01').float()
    b02 = image.select('B02').float()
    b3n = image.select('B3N').float()
    b04 = image.select('B04').float()
    b05 = image.select('B05').float()
    
    # Iron Oxide Index
    iron_index = b02.divide(b01).rename('Iron_Index')
    
    # Ferrous Iron Index
    ferrous_index = b05.divide(b04).rename('Ferrous_Index')
    
    # Ferric Iron Index
    ferric_index = b3n.divide(b01).rename('Ferric_Index')
    
    return image.addBands([iron_index, ferrous_index, ferric_index])

# Apply to both time periods
aster_2001_2006_iron = calculate_iron_index(aster_2001_2006)
aster_2023_2024_iron = calculate_iron_index(aster_2023_2024)

# Calculate data-driven visualization parameters (2nd-98th percentile)
iron_viz_2001 = calculate_viz_params(aster_2001_2006_iron, 'Iron_Index', study_area_buffered)
iron_viz_2023 = calculate_viz_params(aster_2023_2024_iron, 'Iron_Index', study_area_buffered)

# Visualize iron indices with data-driven ranges
Map.addLayer(aster_2001_2006_iron.select('Iron_Index'), {
    'min': iron_viz_2001['min'],
    'max': iron_viz_2001['max'],
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}, 'Iron Index (2001-2006)')

Map.addLayer(aster_2023_2024_iron.select('Iron_Index'), {
    'min': iron_viz_2023['min'],
    'max': iron_viz_2023['max'],
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}, 'Iron Index (2023-2024)')

print("Iron oxide indices calculated!")
Map


Iron oxide indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [74]:
# Carbonate Index using ASTER bands
# Carbonates have characteristic absorption in SWIR bands
# Carbonate Index = B06 / B07

def calculate_carbonate_index(image):
    """Calculate Carbonate Index"""
    b05 = image.select('B05').float()
    b06 = image.select('B06').float()
    b07 = image.select('B07').float()
    b08 = image.select('B08').float()
    
    # Carbonate Index
    carbonate_index = b06.divide(b07).rename('Carbonate_Index')
    
    # Calcite Index
    calcite_index = b08.divide(b07).rename('Calcite_Index')
    
    # Dolomite Index
    dolomite_index = b05.divide(b06).rename('Dolomite_Index')
    
    return image.addBands([carbonate_index, calcite_index, dolomite_index])

# Apply to both time periods
aster_2001_2006_carbonate = calculate_carbonate_index(aster_2001_2006)
aster_2023_2024_carbonate = calculate_carbonate_index(aster_2023_2024)

# Calculate data-driven visualization parameters (2nd-98th percentile)
carbonate_viz_2001 = calculate_viz_params(aster_2001_2006_carbonate, 'Carbonate_Index', study_area_buffered)
carbonate_viz_2023 = calculate_viz_params(aster_2023_2024_carbonate, 'Carbonate_Index', study_area_buffered)

# Visualize carbonate indices with data-driven ranges
Map.addLayer(aster_2001_2006_carbonate.select('Carbonate_Index'), {
    'min': carbonate_viz_2001['min'],
    'max': carbonate_viz_2001['max'],
    'palette': ['blue', 'cyan', 'white', 'pink', 'red']
}, 'Carbonate Index (2001-2006)')

Map.addLayer(aster_2023_2024_carbonate.select('Carbonate_Index'), {
    'min': carbonate_viz_2023['min'],
    'max': carbonate_viz_2023['max'],
    'palette': ['blue', 'cyan', 'white', 'pink', 'red']
}, 'Carbonate Index (2023-2024)')

print("Carbonate indices calculated!")
Map


Carbonate indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Composite Alteration Map


In [80]:
# Create Alteration-Based Mineralization Potential Map
# This combines alteration indices to identify potential mineralization zones
# NOTE: This is based on spectral signatures only (alteration mapping)
# For final mineralization maps incorporating structural controls, see mining-integrated.ipynb

def create_alteration_potential_map(clay_img, iron_img, carbonate_img, study_area):
    """
    Create alteration-based mineralization potential map
    Uses min/max normalization with null handling
    """
    # Get the index bands
    clay_index = clay_img.select('Clay_Index')
    iron_index = iron_img.select('Iron_Index')
    carbonate_index = carbonate_img.select('Carbonate_Index')
    
    # Calculate min/max for normalization - get values client-side first to validate
    print("  Calculating clay index statistics...")
    clay_minmax = clay_index.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    print("  Calculating iron index statistics...")
    iron_minmax = iron_index.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    print("  Calculating carbonate index statistics...")
    carbonate_minmax = carbonate_index.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    # Get min/max values with validation and fallbacks
    clay_min_val = clay_minmax.get('Clay_Index_min')
    clay_max_val = clay_minmax.get('Clay_Index_max')
    iron_min_val = iron_minmax.get('Iron_Index_min')
    iron_max_val = iron_minmax.get('Iron_Index_max')
    carbonate_min_val = carbonate_minmax.get('Carbonate_Index_min')
    carbonate_max_val = carbonate_minmax.get('Carbonate_Index_max')
    
    # Validate and provide fallbacks if null
    if clay_min_val is None or clay_max_val is None:
        print("  Warning: Clay index stats are null, using image statistics...")
        # Use image statistics as fallback
        clay_stats = clay_index.reduceRegion(
            reducer=ee.Reducer.percentile([1, 99]),
            geometry=study_area,
            scale=30,
            maxPixels=1e9
        ).getInfo()
        clay_min_val = clay_stats.get('Clay_Index_p1', 0.5)
        clay_max_val = clay_stats.get('Clay_Index_p99', 1.5)
    
    if iron_min_val is None or iron_max_val is None:
        print("  Warning: Iron index stats are null, using image statistics...")
        iron_stats = iron_index.reduceRegion(
            reducer=ee.Reducer.percentile([1, 99]),
            geometry=study_area,
            scale=30,
            maxPixels=1e9
        ).getInfo()
        iron_min_val = iron_stats.get('Iron_Index_p1', 0.5)
        iron_max_val = iron_stats.get('Iron_Index_p99', 1.5)
    
    if carbonate_min_val is None or carbonate_max_val is None:
        print("  Warning: Carbonate index stats are null, using image statistics...")
        carbonate_stats = carbonate_index.reduceRegion(
            reducer=ee.Reducer.percentile([1, 99]),
            geometry=study_area,
            scale=30,
            maxPixels=1e9
        ).getInfo()
        carbonate_min_val = carbonate_stats.get('Carbonate_Index_p1', 0.5)
        carbonate_max_val = carbonate_stats.get('Carbonate_Index_p99', 1.5)
    
    # Convert to float and ensure valid values
    clay_min = float(clay_min_val) if clay_min_val is not None else 0.5
    clay_max = float(clay_max_val) if clay_max_val is not None else 1.5
    iron_min = float(iron_min_val) if iron_min_val is not None else 0.5
    iron_max = float(iron_max_val) if iron_max_val is not None else 1.5
    carbonate_min = float(carbonate_min_val) if carbonate_min_val is not None else 0.5
    carbonate_max = float(carbonate_max_val) if carbonate_max_val is not None else 1.5
    
    # Ensure min < max
    if clay_min >= clay_max:
        clay_max = clay_min + 0.1
    if iron_min >= iron_max:
        iron_max = iron_min + 0.1
    if carbonate_min >= carbonate_max:
        carbonate_max = carbonate_min + 0.1
    
    print(f"  Clay range: {clay_min:.4f} to {clay_max:.4f}")
    print(f"  Iron range: {iron_min:.4f} to {iron_max:.4f}")
    print(f"  Carbonate range: {carbonate_min:.4f} to {carbonate_max:.4f}")
    
    # Normalize each index to 0-1 range using unitScale with validated values
    clay_prob = clay_index.unitScale(clay_min, clay_max).clamp(0, 1)
    iron_prob = iron_index.unitScale(iron_min, iron_max).clamp(0, 1)
    carbonate_prob = carbonate_index.unitScale(carbonate_min, carbonate_max).clamp(0, 1)
    
    # Weighted combination
    alteration_potential = clay_prob.multiply(0.4).add(
        iron_prob.multiply(0.3)
    ).add(
        carbonate_prob.multiply(0.3)
    ).rename('Alteration_Potential')
    
    # Classify into zones using percentile-based thresholds
    print("  Calculating percentile thresholds for classification...")
    stats = alteration_potential.reduceRegion(
        reducer=ee.Reducer.percentile([70, 80, 90, 95]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    # Get percentile thresholds with validation
    p70_val = stats.get('Alteration_Potential_p70', 0.7)
    p80_val = stats.get('Alteration_Potential_p80', 0.8)
    p90_val = stats.get('Alteration_Potential_p90', 0.9)
    p95_val = stats.get('Alteration_Potential_p95', 0.95)
    
    p70 = float(p70_val) if p70_val is not None else 0.7
    p80 = float(p80_val) if p80_val is not None else 0.8
    p90 = float(p90_val) if p90_val is not None else 0.9
    p95 = float(p95_val) if p95_val is not None else 0.95
    
    # Convert back to server-side for classification
    p70 = ee.Number(p70)
    p80 = ee.Number(p80)
    p90 = ee.Number(p90)
    p95 = ee.Number(p95)
    
    # Classify into confidence levels
    very_high = alteration_potential.gte(p95).rename('Very_High_Confidence_Zones')
    high = alteration_potential.gte(p90).And(alteration_potential.lt(p95)).rename('High_Confidence_Zones')
    medium = alteration_potential.gte(p80).And(alteration_potential.lt(p90)).rename('Medium_Confidence_Zones')
    low = alteration_potential.gte(p70).And(alteration_potential.lt(p80)).rename('Low_Confidence_Zones')
    all_zones = alteration_potential.gte(p70).rename('All_Alteration_Zones')
    alteration_zones = alteration_potential.gte(p90).rename('Alteration_Zones')
    
    return alteration_potential.addBands([
        very_high, high, medium, low, all_zones, alteration_zones
    ])

# Create alteration-based potential maps for both periods
print("Creating alteration potential maps...")
print("This may take a moment as it calculates statistics for normalization...")
print("\nProcessing 2001-2006 period...")
alteration_potential_2001_2006 = create_alteration_potential_map(
    aster_2001_2006_clay,
    aster_2001_2006_iron,
    aster_2001_2006_carbonate,
    study_area_buffered
)

print("\nProcessing 2023-2024 period...")
alteration_potential_2023_2024 = create_alteration_potential_map(
    aster_2023_2024_clay,
    aster_2023_2024_iron,
    aster_2023_2024_carbonate,
    study_area_buffered
)

# Keep legacy variable names for backward compatibility with integrated notebook
mineralisation_2023_2024 = alteration_potential_2023_2024.select('Alteration_Potential').rename('Mineralisation_Probability')
mineralisation_2001_2006 = alteration_potential_2001_2006.select('Alteration_Potential').rename('Mineralisation_Probability')

# Visualize Alteration-Based Potential with improved confidence levels
print("\n" + "=" * 60)
print("ALTERATION-BASED MINERALIZATION POTENTIAL")
print("=" * 60)
print("Using data-driven normalization (min/max for each index)")
print("Using percentile-based thresholds (Top 5%, 10%, 20%, 30%)")
print("This represents potential based on SPECTRAL ALTERATION ONLY")
print("For final mineralization maps with structural controls, see mining-integrated.ipynb")
print("=" * 60)

# Calculate visualization parameters for alteration potential (2nd-98th percentile)
print("Calculating visualization parameters for Alteration Potential...")

# Calculate percentiles directly for visualization
alteration_band = alteration_potential_2023_2024.select('Alteration_Potential')
viz_stats = alteration_band.reduceRegion(
    reducer=ee.Reducer.percentile([2, 98]),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
)

# Get the actual values with robust error handling
try:
    stats_dict = viz_stats.getInfo()
    print(f"Available keys in stats: {list(stats_dict.keys())}")
    
    # Try the correct key format first
    min_key = 'Alteration_Potential_p2'
    max_key = 'Alteration_Potential_p98'
    
    alteration_viz_min = stats_dict.get(min_key)
    alteration_viz_max = stats_dict.get(max_key)
    
    # Try alternative key formats if needed
    if alteration_viz_min is None:
        alteration_viz_min = stats_dict.get('AlterationPotential_p2') or stats_dict.get('p2')
    if alteration_viz_max is None:
        alteration_viz_max = stats_dict.get('AlterationPotential_p98') or stats_dict.get('p98')
    
    # Convert to float with validation
    if alteration_viz_min is not None:
        alteration_viz_min = float(alteration_viz_min)
    else:
        alteration_viz_min = 0.0
        print("Warning: Using default min = 0.0")
    
    if alteration_viz_max is not None:
        alteration_viz_max = float(alteration_viz_max)
    else:
        alteration_viz_max = 1.0
        print("Warning: Using default max = 1.0")
    
    # Ensure min < max
    if alteration_viz_min >= alteration_viz_max:
        alteration_viz_max = alteration_viz_min + 0.1
        print(f"Adjusted max to {alteration_viz_max}")
        
except Exception as e:
    print(f"Error: {e}")
    alteration_viz_min = 0.0
    alteration_viz_max = 1.0

# Final validation - ensure values are definitely valid numbers
alteration_viz_min = float(alteration_viz_min) if alteration_viz_min is not None else 0.0
alteration_viz_max = float(alteration_viz_max) if alteration_viz_max is not None else 1.0

# Ensure they're not NaN or inf
import math
if math.isnan(alteration_viz_min) or math.isinf(alteration_viz_min):
    alteration_viz_min = 0.0
if math.isnan(alteration_viz_max) or math.isinf(alteration_viz_max):
    alteration_viz_max = 1.0

print(f"Final visualization range: {alteration_viz_min:.4f} to {alteration_viz_max:.4f}")

# Continuous potential map with data-driven range (2023-2024)
Map.addLayer(
    alteration_potential_2023_2024.select('Alteration_Potential'), 
    {
        'min': float(alteration_viz_min),
        'max': float(alteration_viz_max),
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 
    'Alteration-Based Potential (2023-2024) - Data-driven range'
)

# Continuous potential map for 2001-2006 (for comparison)
# Calculate viz params for 2001-2006
alteration_viz_2001_stats = alteration_potential_2001_2006.select('Alteration_Potential').reduceRegion(
    reducer=ee.Reducer.percentile([2, 98]),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo()

alteration_viz_2001_min = float(alteration_viz_2001_stats.get('Alteration_Potential_p2', 0.0))
alteration_viz_2001_max = float(alteration_viz_2001_stats.get('Alteration_Potential_p98', 1.0))

if alteration_viz_2001_min >= alteration_viz_2001_max:
    alteration_viz_2001_max = alteration_viz_2001_min + 0.1

Map.addLayer(
    alteration_potential_2001_2006.select('Alteration_Potential'), 
    {
        'min': alteration_viz_2001_min,
        'max': alteration_viz_2001_max,
        'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
    }, 
    'Alteration-Based Potential (2001-2006) - Data-driven range', 
    False
)

# Very High Confidence (Top 5% - Most Significant Targets)
Map.addLayer(alteration_potential_2023_2024.select('Very_High_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'darkred']
}, 'Very High Confidence (Top 5%)', False)

# High Confidence (Top 10%)
Map.addLayer(alteration_potential_2023_2024.select('High_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'red']
}, 'High Confidence (Top 10%)', False)

# Medium Confidence (Top 20%)
Map.addLayer(alteration_potential_2023_2024.select('Medium_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'orange']
}, 'Medium Confidence (Top 20%)', False)

# Low Confidence (Top 30%)
Map.addLayer(alteration_potential_2023_2024.select('Low_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'yellow']
}, 'Low Confidence (Top 30%)', False)

# All Zones Combined (Top 30% - for overview)
Map.addLayer(alteration_potential_2023_2024.select('All_Alteration_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'red']
}, 'All Alteration Zones (Top 30%)', False)

print("\n✓ Alteration-based potential maps created!")
print("\nConfidence Levels (Percentile-based):")
print("  🔴 Very High (Dark Red): Top 5% - Most significant alteration")
print("  🔴 High (Red): Top 10% - High confidence alteration")
print("  🟠 Medium (Orange): Top 20% - Moderate confidence")
print("  🟡 Low (Yellow): Top 30% - Potential alteration")
print("\n⚠️  IMPORTANT:")
print("  - These maps show ALTERATION-BASED potential only (spectral signatures)")
print("  - They do NOT incorporate structural controls (lineaments, faults)")
print("  - For FINAL MINERALIZATION MAPS with structural controls,")
print("    see mining-integrated.ipynb (standard practice in literature)")
print("=" * 60)

# Display the map
Map

Creating alteration potential maps...
This may take a moment as it calculates statistics for normalization...

Processing 2001-2006 period...
  Calculating clay index statistics...


  Calculating iron index statistics...
  Calculating carbonate index statistics...
  Clay range: 0.8151 to 1.5319
  Iron range: 0.4250 to 1.0514
  Carbonate range: 0.6296 to 2.5385
  Calculating percentile thresholds for classification...

Processing 2023-2024 period...
  Calculating clay index statistics...
  Calculating iron index statistics...
  Calculating carbonate index statistics...
  Clay range: 0.5000 to 1.5000
  Iron range: 0.4322 to 1.0781
  Carbonate range: 0.5000 to 1.5000
  Calculating percentile thresholds for classification...

ALTERATION-BASED MINERALIZATION POTENTIAL
Using data-driven normalization (min/max for each index)
Using percentile-based thresholds (Top 5%, 10%, 20%, 30%)
This represents potential based on SPECTRAL ALTERATION ONLY
For final mineralization maps with structural controls, see mining-integrated.ipynb
Calculating visualization parameters for Alteration Potential...
Available keys in stats: ['Alteration_Potential_p2', 'Alteration_Potential_p98']
Fin

EEException: Image.visualize: Color is not a valid CSS 3.0 color ('FF0000' or 'red' for red). Found: 'transparent'.

In [67]:
# Create composite alteration maps combining all three mineral types
def create_alteration_composite(clay_img, iron_img, carbonate_img):
    """Combine clay, iron, and carbonate indices into RGB composite"""
    # Normalize indices to 0-255 range for visualization
    clay_norm = clay_img.select('Clay_Index').multiply(255).byte()
    iron_norm = iron_img.select('Iron_Index').multiply(255).byte()
    carbonate_norm = carbonate_img.select('Carbonate_Index').multiply(255).byte()
    
    # Create RGB composite: R=Iron, G=Clay, B=Carbonate
    composite = ee.Image.cat([iron_norm, clay_norm, carbonate_norm]).rename(['R', 'G', 'B'])
    
    return composite

# Create composites for both periods
alteration_2001_2006 = create_alteration_composite(
    aster_2001_2006_clay,
    aster_2001_2006_iron,
    aster_2001_2006_carbonate
)

alteration_2023_2024 = create_alteration_composite(
    aster_2023_2024_clay,
    aster_2023_2024_iron,
    aster_2023_2024_carbonate
)

# Calculate data-driven visualization parameters for RGB composite bands (2nd-98th percentile)
alteration_rgb_viz_2001 = calculate_rgb_viz_params(alteration_2001_2006, ['R', 'G', 'B'], study_area_buffered)
alteration_rgb_viz_2023 = calculate_rgb_viz_params(alteration_2023_2024, ['R', 'G', 'B'], study_area_buffered)

# Visualize composite alteration maps with data-driven ranges
Map.addLayer(alteration_2001_2006, {
    'min': alteration_rgb_viz_2001['min'],
    'max': alteration_rgb_viz_2001['max']
}, 'Alteration Composite (2001-2006) - R:Iron G:Clay B:Carbonate')

Map.addLayer(alteration_2023_2024, {
    'min': alteration_rgb_viz_2023['min'],
    'max': alteration_rgb_viz_2023['max']
}, 'Alteration Composite (2023-2024) - R:Iron G:Clay B:Carbonate')

print("Composite alteration maps created!")
Map


EEException: Number.min: Parameter 'right' is required and may not be null.

## Mineralisation Analysis


In [68]:
# Create mineralisation probability map
# Combine all alteration indices to identify potential mineralisation zones

def create_mineralisation_map(clay_img, iron_img, carbonate_img):
    """Create mineralisation probability map"""
    # Normalize and combine indices
    clay_prob = clay_img.select('Clay_Index').subtract(0.8).multiply(10).clamp(0, 1)
    iron_prob = iron_img.select('Iron_Index').subtract(0.8).multiply(5).clamp(0, 1)
    carbonate_prob = carbonate_img.select('Carbonate_Index').subtract(0.9).multiply(10).clamp(0, 1)
    
    # Weighted combination (adjust weights as needed)
    mineralisation = clay_prob.multiply(0.4).add(
        iron_prob.multiply(0.3)
    ).add(
        carbonate_prob.multiply(0.3)
    ).rename('Mineralisation_Probability')
    
    # Classify into zones
    mineralisation_zones = mineralisation.gt(0.5).rename('Mineralisation_Zones')
    
    return mineralisation.addBands(mineralisation_zones)

# Create mineralisation maps for both periods
mineralisation_2001_2006 = create_mineralisation_map(
    aster_2001_2006_clay,
    aster_2001_2006_iron,
    aster_2001_2006_carbonate
)

mineralisation_2023_2024 = create_mineralisation_map(
    aster_2023_2024_clay,
    aster_2023_2024_iron,
    aster_2023_2024_carbonate
)

# Calculate data-driven visualization parameters (2nd-98th percentile)
mineralisation_viz_2001 = calculate_viz_params(mineralisation_2001_2006, 'Mineralisation_Probability', study_area_buffered)
mineralisation_viz_2023 = calculate_viz_params(mineralisation_2023_2024, 'Mineralisation_Probability', study_area_buffered)

# Visualize mineralisation with data-driven ranges
Map.addLayer(mineralisation_2001_2006.select('Mineralisation_Probability'), {
    'min': mineralisation_viz_2001['min'],
    'max': mineralisation_viz_2001['max'],
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Mineralisation Probability (2001-2006)')

# Binary zones use fixed 0-1 range (appropriate for binary data)
Map.addLayer(mineralisation_2001_2006.select('Mineralisation_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']
}, 'Mineralisation Zones (2001-2006)')

Map.addLayer(mineralisation_2023_2024.select('Mineralisation_Probability'), {
    'min': mineralisation_viz_2023['min'],
    'max': mineralisation_viz_2023['max'],
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Mineralisation Probability (2023-2024)')

# Binary zones use fixed 0-1 range (appropriate for binary data)
Map.addLayer(mineralisation_2023_2024.select('Mineralisation_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']
}, 'Mineralisation Zones (2023-2024)')

print("Mineralisation maps created!")
Map


Mineralisation maps created!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Temporal Change Detection


In [ ]:
# Calculate change between the two time periods
def calculate_change(period1, period2):
    """Calculate change between two periods"""
    change = period2.subtract(period1).rename('Change')
    change_magnitude = change.abs().rename('Change_Magnitude')
    return change.addBands(change_magnitude)

# Calculate changes for each mineral type
clay_change = calculate_change(
    aster_2001_2006_clay.select('Clay_Index'),
    aster_2023_2024_clay.select('Clay_Index')
)

iron_change = calculate_change(
    aster_2001_2006_iron.select('Iron_Index'),
    aster_2023_2024_iron.select('Iron_Index')
)

carbonate_change = calculate_change(
    aster_2001_2006_carbonate.select('Carbonate_Index'),
    aster_2023_2024_carbonate.select('Carbonate_Index')
)

mineralisation_change = calculate_change(
    mineralisation_2001_2006.select('Mineralisation_Probability'),
    mineralisation_2023_2024.select('Mineralisation_Probability')
)

# Calculate data-driven visualization parameters for change maps (2nd-98th percentile)
# For change maps, we use symmetric percentiles to show both positive and negative changes
clay_change_viz = calculate_viz_params(clay_change, 'Change', study_area_buffered)
iron_change_viz = calculate_viz_params(iron_change, 'Change', study_area_buffered)
carbonate_change_viz = calculate_viz_params(carbonate_change, 'Change', study_area_buffered)
mineralisation_change_viz = calculate_viz_params(mineralisation_change, 'Change', study_area_buffered)

# Visualize changes with data-driven ranges
Map.addLayer(clay_change.select('Change'), {
    'min': clay_change_viz['min'],
    'max': clay_change_viz['max'],
    'palette': ['blue', 'white', 'red']
}, 'Clay Change (2023-2024 vs 2001-2006)')

Map.addLayer(iron_change.select('Change'), {
    'min': iron_change_viz['min'],
    'max': iron_change_viz['max'],
    'palette': ['blue', 'white', 'red']
}, 'Iron Change (2023-2024 vs 2001-2006)')

Map.addLayer(carbonate_change.select('Change'), {
    'min': carbonate_change_viz['min'],
    'max': carbonate_change_viz['max'],
    'palette': ['blue', 'white', 'red']
}, 'Carbonate Change (2023-2024 vs 2001-2006)')

Map.addLayer(mineralisation_change.select('Change'), {
    'min': mineralisation_change_viz['min'],
    'max': mineralisation_change_viz['max'],
    'palette': ['blue', 'white', 'red']
}, 'Mineralisation Change (2023-2024 vs 2001-2006)')

print("Temporal change analysis completed!")
Map


Temporal change analysis completed!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Export Results


In [ ]:
# Export alteration and mineralisation maps
# Note: Uncomment and run to export to Google Drive

export_params = {
    'image': None,  # Will be set for each export
    'description': None,  # Will be set for each export
    'scale': 30,  # 30m resolution
    'region': study_area_buffered,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13
}

# Export functions (uncomment to use)
def export_alteration_maps():
    """Export all alteration maps"""
    # Period 1
    geemap.ee_export_image(
        aster_2001_2006_clay.select(['Clay_Index', 'AlOH_Index', 'Kaolinite_Index']),
        filename='aster_alteration_clay_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2001_2006_iron.select(['Iron_Index', 'Ferrous_Index', 'Ferric_Index']),
        filename='aster_alteration_iron_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2001_2006_carbonate.select(['Carbonate_Index', 'Calcite_Index', 'Dolomite_Index']),
        filename='aster_alteration_carbonate_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Period 2
    geemap.ee_export_image(
        aster_2023_2024_clay.select(['Clay_Index', 'AlOH_Index', 'Kaolinite_Index']),
        filename='aster_alteration_clay_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2023_2024_iron.select(['Iron_Index', 'Ferrous_Index', 'Ferric_Index']),
        filename='aster_alteration_iron_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2023_2024_carbonate.select(['Carbonate_Index', 'Calcite_Index', 'Dolomite_Index']),
        filename='aster_alteration_carbonate_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

def export_mineralisation_maps():
    """Export mineralisation maps"""
    geemap.ee_export_image(
        mineralisation_2001_2006,
        filename='aster_mineralisation_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        mineralisation_2023_2024,
        filename='aster_mineralisation_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

def export_change_maps():
    """Export change detection maps"""
    geemap.ee_export_image(
        clay_change,
        filename='aster_clay_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        iron_change,
        filename='aster_iron_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        carbonate_change,
        filename='aster_carbonate_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        mineralisation_change,
        filename='aster_mineralisation_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

print("Export functions defined. Uncomment and call to export maps.")
print("Example: export_alteration_maps()")
